# <center>[🦏 Автоматический стекинг и блендинг](https://stepik.org/lesson/872530/)</center>

### Оглавление ноутбука

<img src='../images/pipelines.jpg' align="right" width="600" height="600" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">☘️ Три модели для блендинга </a></li>
<li><a href="#look1">🔧 Построение пайплана</a>
<li><a href="#check1"> 🎓🐊 Обучим ансамбль</a>
<li><a href="#6">🧸 Выводы и заключения</a>

</li></ul></font></p>


<div class="alert alert-info">

* Как вы уже могли заметить, чем больше моделей вы `стекаете`, тем больше у вас разрастается код, а количество беспорядка в нем растет по экспоненте.
* Но есть специальные инструменты, которые позволяют сделать это элегантно и даже более эффективно. Да еще и меньшим числом строк кода! 
* Не без ограничений и недостатков, увы.

<div class="alert alert-info">
    
Сегодня мы поговорим про `sklearn.Pipelines` - способ упаковать ваш процесс обучения и инференса от `Feature Engineering` до стекинга 10 моделей в один пайплайн.

## Импортируем библиотеки

In [2]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import lightgbm as lgbm
import xgboost as xgb
import catboost as cb

In [3]:
# pip install xgboost -U -q

In [4]:
import warnings
warnings.filterwarnings("ignore")

## Считываем данные

In [5]:
from sklearn import preprocessing
data = pd.read_csv('../data/quickstart_train.csv')

categorical_features = ['model', 'car_type', 'fuel_type']

for cat in categorical_features:
    lbl = preprocessing.LabelEncoder()
    data[cat] = lbl.fit_transform(data[cat].astype(str))
    data[cat] = data[cat].astype('category')
    
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype   
---  ------                    --------------  -----   
 0   car_id                    2337 non-null   object  
 1   model                     2337 non-null   category
 2   car_type                  2337 non-null   category
 3   fuel_type                 2337 non-null   category
 4   car_rating                2337 non-null   float64 
 5   year_to_start             2337 non-null   int64   
 6   riders                    2337 non-null   int64   
 7   year_to_work              2337 non-null   int64   
 8   target_reg                2337 non-null   float64 
 9   target_class              2337 non-null   object  
 10  mean_rating               2337 non-null   float64 
 11  distance_sum              2337 non-null   float64 
 12  rating_min                2337 non-null   float64 
 13  speed_max                 2337 non-null   float6

### Разделим выборку на валидационную и обучающую

In [6]:
# значения таргета закодируем целыми числами
class_names = np.unique(data['target_class'])
data['target_class'] = data['target_class'].replace(class_names, np.arange(data['target_class'].nunique()))

In [7]:
cols2drop = ['car_id', 'target_reg', 'target_class']
categorical_features = ['model', 'car_type', 'fuel_type']
numerical_features = [c for c in data.columns if c not in categorical_features and c not in cols2drop]

In [8]:
X_train, X_val, y_train, y_val = train_test_split(data.drop(cols2drop, axis=1), 
                                                    data['target_class'],
                                                    test_size=.25,
                                                    stratify=data['target_class'],
                                                    random_state=42)
print(X_train.shape, X_val.shape)

(1752, 14) (585, 14)


# <center> ☘️ Объявим 3 модели

#  😺🚀 Модель `CatBoost`

In [9]:
params_cat = {
             'n_estimators' : 700,
              # 'learning_rate': .03,
              'depth' : 3,
              'verbose': False,
              'use_best_model': True,
              'cat_features' : categorical_features,
              'text_features': [],
              # 'train_dir' : '/home/jovyan/work/catboost',
              'border_count' : 64,
              'l2_leaf_reg' : 1,
              'bagging_temperature' : 2,
              'rsm' : 0.51,
              'loss_function': 'MultiClass',
              'auto_class_weights' : 'Balanced', #try not balanced
              'random_state': 42,
              'use_best_model': False,
              # 'custom_metric' : ['AUC', 'MAP'] # Не работает внутри sklearn.Pipelines
         }

In [10]:
cat_model = cb.CatBoostClassifier(**params_cat)

# 🦄🎳 Модель `LightGBM`

In [11]:
categorical_features_index = [i for i in range(data.shape[1]) if data.columns[i] in categorical_features]
params_lgbm = {
    "num_leaves": 200,
    "n_estimators": 1500,
    # "max_depth": 7,
    "min_child_samples": None,
    "learning_rate": 0.001,
    "min_data_in_leaf": 5,
    "feature_fraction": 0.98,
    # "categorical_feature": cat_cols,
    'reg_alpha' : 3.0,
    'reg_lambda' : 5.0,
    'categorical_feature': categorical_features_index
}

In [12]:
lgbm_model = lgbm.LGBMClassifier(**params_lgbm)

# 👽🔱 Модель `XGBoost`

In [13]:
params_xgb = {
    "eta": 0.05,
    'n_estimators' : 1500,
    "max_depth": 6,
    "subsample": 0.7,
    # "colsample_bytree": 0.95,
    'min_child_weight' : 0.1,
    'gamma': .01,
    'reg_lambda' : 0.1,
    'reg_alpha' : 0.5,
    "objective": "reg:linear",
    "eval_metric": "mae",
    'tree_method' : 'hist', # Supported tree methods for cat fs are `gpu_hist`, `approx`, and `hist`.
    'enable_categorical' : True
    
}

In [14]:
xgb_model = xgb.XGBClassifier(**params_xgb)

# <center> 🥤 Построим пайплан

In [19]:
!pip3 install -U scikit-learn==1.2.2

  Using cached scikit_learn-1.2.2-cp310-cp310-macosx_12_0_arm64.whl (8.5 MB)
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.3.0
    Uninstalling scikit-learn-1.3.0:
      Successfully uninstalled scikit-learn-1.3.0

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [20]:
# Вспомогательные блоки организации для пайплайна
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector

In [21]:
# Вспомогательные элементы для наполнения пайплайна
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler

In [22]:
# Некоторые модели для построения ансамбля
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

In [23]:
# Добавим визуализации
import sklearn
sklearn.set_config(display='diagram')

from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

### Предобработаем данные
Под каждый тип данных заводим свой трансформер

In [24]:
# заменяет пропуски самым частым значением и делает ohe
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))])

In [25]:
# заменяет пропуски средним значением и делает нормализацию
numerical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler())
])

In [26]:
# соединим два предыдущих трансформера в один
preprocessor = ColumnTransformer(transformers=[
    ("numerical", numerical_transformer, numerical_features),
    ("categorical", categorical_transformer, categorical_features)])

preprocessor

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['car_rating', 'year_to_start', 'riders',
                                  'year_to_work', 'mean_rating', 'distance_sum',
                                  'rating_min', 'speed_max',
                                  'user_ride_quality_median',
                                  'deviation_normal_count', 'user_uniq']),
                                ('categorical',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['model', 'car_type', 'fuel_type'])])

In [27]:
preprocessor.transformers[0]

('numerical',
 Pipeline(steps=[('imputer', SimpleImputer()), ('scaler', StandardScaler())]),
 ['car_rating',
  'year_to_start',
  'riders',
  'year_to_work',
  'mean_rating',
  'distance_sum',
  'rating_min',
  'speed_max',
  'user_ride_quality_median',
  'deviation_normal_count',
  'user_uniq'])

# <center> 🎓🐊 Обучим ансамбль

In [28]:
# список базовых моделей
estimators = [
    
    
    ("ExtraTrees",  make_pipeline(preprocessor, ExtraTreesClassifier(n_estimators = 10_000, max_depth = 6, min_samples_leaf = 2, 
                                                              bootstrap = True, class_weight = 'balanced', # ccp_alpha = 0.001, 
                                                              random_state = 75, verbose=False, n_jobs=-1,))),
    

    ("XGBoost", xgb_model),
    ("LightGBM", lgbm_model),
    ("CatBoost", cat_model),
    
    # То, что не дало прироста в ансамбле
    # ("SVM", make_pipeline(preprocessor, LinearSVC(verbose=False))),
    # ("MLP", make_pipeline(preprocessor, MLPClassifier(verbose=False, hidden_layer_sizes=(100, 30, ), alpha=0.001,random_state=75, max_iter = 1300, ))),
    ("Random_forest",  make_pipeline(preprocessor, RandomForestClassifier(n_estimators = 15_000, max_depth = 7, 
                                                              min_samples_leaf = 2,
                                                              warm_start = True, n_jobs=-1,
                                                              random_state = 75, verbose=False))),
    
    
    
]

# в качестве мета-модели будем использовать LogisticRegression
meta_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(verbose=False),
    # final_estimator=RandomForestClassifier(n_estimators = 10_000, 
                                           # max_depth = 5,
                                           # verbose=False),
    n_jobs=-1,
    verbose=False,
)

stacking_classifier = meta_model
stacking_classifier

StackingClassifier(estimators=[('ExtraTrees',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('numerical',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer()),
                                                                                                  ('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['car_rating',
                                                                                   'year_to_start',
                                                                                   'riders',
                                                                                   'year_to_work',
                                                                                   'mean_rating',
                                                                                   'distance_sum',
                                                                                   'rating_min',
                                                                                   'speed_max',
                                                                                   'user_ride_quality_median',
                                                                                   'deviation_nor...
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('onehot',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['model',
                                                                                   'car_type',
                                                                                   'fuel_type'])])),
                                                ('randomforestclassifier',
                                                 RandomForestClassifier(max_depth=7,
                                                                        min_samples_leaf=2,
                                                                        n_estimators=15000,
                                                                        n_jobs=-1,
                                                                        random_state=75,
                                                                        verbose=False,
                                                                        warm_start=True))]))],
                   final_estimator=LogisticRegression(verbose=False), n_jobs=-1,
                   verbose=False)

In [29]:
stacking_classifier.fit(X_train, y_train)

/Users/sergak/miniconda3/lib/python3.10/site-packages/lightgbm/basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
/Users/sergak/miniconda3/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')
/Users/sergak/miniconda3/lib/python3.10/site-packages/lightgbm/basic.py:1487: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(f'{key} keyword has been found in `params` and will be ignored.\n'
/Users/sergak/miniconda3/lib/python3.10/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_feature in param 

[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98
[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98
[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98
[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98
[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98
[LightGBM] [Warning] feature_fraction is set=0.98, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.98


StackingClassifier(estimators=[('ExtraTrees',
                                Pipeline(steps=[('columntransformer',
                                                 ColumnTransformer(transformers=[('numerical',
                                                                                  Pipeline(steps=[('imputer',
                                                                                                   SimpleImputer()),
                                                                                                  ('scaler',
                                                                                                   StandardScaler())]),
                                                                                  ['car_rating',
                                                                                   'year_to_start',
                                                                                   'riders',
                                                                                   'year_to_work',
                                                                                   'mean_rating',
                                                                                   'distance_sum',
                                                                                   'rating_min',
                                                                                   'speed_max',
                                                                                   'user_ride_quality_median',
                                                                                   'deviation_nor...
                                                                                                   SimpleImputer(strategy='most_frequent')),
                                                                                                  ('onehot',
                                                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                                                  ['model',
                                                                                   'car_type',
                                                                                   'fuel_type'])])),
                                                ('randomforestclassifier',
                                                 RandomForestClassifier(max_depth=7,
                                                                        min_samples_leaf=2,
                                                                        n_estimators=15000,
                                                                        n_jobs=-1,
                                                                        random_state=75,
                                                                        verbose=False,
                                                                        warm_start=True))]))],
                   final_estimator=LogisticRegression(verbose=False), n_jobs=-1,
                   verbose=False)

In [30]:
# for i in stacking_classifier.estimators:
#     print(i[0])

In [31]:
# dir(stacking_classifier)

In [32]:
corr_df = pd.DataFrame()

for model, (name, _) in zip(stacking_classifier.estimators_, stacking_classifier.estimators):
    preprocessed = stacking_classifier.estimators[0][1].steps[0][1].fit(X_train, y_train).transform(X_val)
    print(name, 'accuracy: ', round(accuracy_score(model.predict(X_val), y_val), 4))
    
    corr_df[name] = model.predict(X_val)
    

ExtraTrees accuracy:  0.7368
XGBoost accuracy:  0.7949
LightGBM accuracy:  0.8085
CatBoost accuracy:  0.8034
Random_forest accuracy:  0.7487


In [33]:
corr_df.corr().style.background_gradient(cmap="RdYlGn")

,ExtraTrees,XGBoost,LightGBM,CatBoost,Random_forest
ExtraTrees,1.000000,0.856435,0.851740,0.855011,0.841299
XGBoost,0.856435,1.000000,0.976487,0.953732,0.946774
LightGBM,0.851740,0.976487,1.000000,0.969285,0.953330
CatBoost,0.855011,0.953732,0.969285,1.000000,0.948848
Random_forest,0.841299,0.946774,0.953330,0.948848,1.000000


In [34]:
# Random_forest сильно коррелирует с другими моделями, 
# поэтому он снижает качество ансамбля.
# Попробуйте его убрать

In [35]:
print('ensemble score:', round(accuracy_score(stacking_classifier.predict(X_val), y_val), 4))
# как вы можете заметить, ансамбль довольно заметно улучшил качество решения

ensemble score: 0.8051


# Комментарии

* 📈 Да, скор ансамбля вырос, но есть много **"но"** у этой реализации
* ⚠️ Тут в качестве мета-модели использовалась `LogisticRegression`, что по сути является обычным блендингом, но с кросс-валидацией.
* 🧩 Слабые или похожие модели мешают ансамблю поднять скор (Если убрать `RandomForest`, то скор поднимется)
* 🍏 Стекинг можно усложнить, подавая мета-модели еще признаки, при этом используя более сложную meta-модель.
* 🤔 Тогда в зависимости от свойств объекта, мета-модели, такие как `RandomForestClassifier`, могут принимать решение оптимальнее.
* ☹️ В рамках `pipeline` в `sklearn` это сделать сложнее. Надо взять что-то другое. 

* Не все можно запихнуть в `pipeline`. Например, `eval_set` для `early-stopping` или класс `train` от `LightGBM`
* Что есть еще? Об этом позже

# <center> 📚 Дополнительная литература
- [Статья Александра Дьяконова про Стекинг и Блендинг](https://alexanderdyakonov.wordpress.com/2017/03/10/c%D1%82%D0%B5%D0%BA%D0%B8%D0%BD%D0%B3-stacking-%D0%B8-%D0%B1%D0%BB%D0%B5%D0%BD%D0%B4%D0%B8%D0%BD%D0%B3-blending/)
- [Пример решения, где  `LightGBM` в качестве мета-модели](https://www.youtube.com/watch?v=aMlpeDOjib8)

# <center> 🧸 Выводы и заключения

<div class="alert alert-info">

* `sklearn.Pipelines` - это очень сильный инструмент, позволяющий упаковать весь процесс обучения модели в один механизм
* Легко добавлять новые модели и которые легко применять на инференсе.
* Это один из тех инструментов, который часто используется не только на соревнованиях, но и в обычной работе из-за своей элегантности и простоты.